In [1]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy as np
from collections import deque

In [245]:
paths_common = ['GESTION/NOJO',
                'GESTION/TETIER_R4',
                'GESTION/NOM_HTML',
                'DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/IDENT/TEL',
                'DONNEES/OBJET/OBJET_COMPLET'
                ]

#give_me_the_tag = ['DONNEES/INFO',
                   #'DONNEES/TYPE_ACTIVITE_ORG']

paths_take_all = ['DONNEES/CLASSES',
                  'DONNEES/DESCRIPTEURS',
                  'DONNEES/CARACTERISTIQUES']

jump_a_line = ['DONNEES/PROCEDURES/LOTS']

tags = []
for path in paths_common:
    tags.append(path2tag(path))
for path in paths_take_all:
    tags.append(path2tag(path))
tags.append('LOTS')

In [246]:
def path2tag(path):
    pavel = ''
    for i in reversed(path):
        if i != '/':
            pavel += i
        if i == '/':
            break
    return pavel[::-1]

In [249]:
def give_me_your_dict(annonce):
    lots = []
    out = []
    if annonce.find('DONNEES/PROCEDURES/LOTS') is not None:
        reslit = list(annonce.find('DONNEES/PROCEDURES/LOTS').iter())
    else:
        return out
    
    if reslit is not None:
        for element in reslit:
            if element.tag == 'NUM_LOT' and element.text is not None:
                sub_list = []
                for lot_element in reslit[reslit.index(element):]:
                    sub_list.append(lot_element)
                    if reslit.index(lot_element) + 1 < len(reslit):
                        next_element = reslit[reslit.index(lot_element) + 1]
                
                        if next_element.tag == 'NUM_LOT' and next_element.text is not None:
                            lots.append(sub_list)
                            break
    for arr in lots:
        out.append({element.tag: element.text for element in arr})
        
    return out

In [224]:
def xml_parser(paths_common, paths_take_all, file):
    rowsamarr = []
    rows = []
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                rows.append(element.text)
            else:
                rows.append('')
        elif element is None:
            rows.append('')
    """
    for path in give_me_the_tag:
        element = file.find(path)
        pavel = ''
        if element is not None:
            for child in element.getchildren():
                if child is not None:
                    pavel += child.tag
            pairs.append([path2tag(path), pavel])
        elif element is None:
            pairs.append([path2tag(path), ''])
    """     
    
    for path in paths_take_all:
        element = file.find(path)
        if element is not None:
            reslist = list(element.iter()) 
            result = ' '.join([element.text for element in reslist if element.text is not None])
            rows.append(result)
        elif element is None:
            rows.append('')
    
    lots = give_me_your_dict(file)
    if len(lots) > 0:
        rows.append(lots[0])
        rowsamarr.append(rows)
        if len(lots) > 1:
            for lot in lots[1:]:
                row = ['' for i in range(len(tags) - 1)]
                row.append(lot)
                rowsamarr.append(row)
    else:
        rowsamarr.append(rows)
        
    return rowsamarr, tags 


In [243]:
def write_to_csv(big_row_list, file_name, tags, year):
    bar3 = pbar(len(big_row_list))
    
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=';', quotechar=' ',
                            quoting=csv.QUOTE_ALL)
        spamwriter.writerow(tags)
        print('{} rows for year {}'.format(len(big_row_list), year))
        for row in big_row_list:
            for sub_row in row:
                spamwriter.writerow(sub_row)
        
            

In [228]:
def pbar(maxlen):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [229]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]
    bar3 = pbar(len(years))

    
    for i in range(len(years)):
        bar3.update(i)
        paths = glob.glob('{}/mp*/*.xml'.format(years[i]))
        #paths = glob.glob('rand/*.xml')
        annonces = []
    
        bar = pbar(len(paths))
        
    
        for path in paths:
            bar.update(paths.index(path))
            file = open(path, encoding='ISO-8859-1')
            apple = file.read()
            
            apple = apple.replace('&apos;', "'")
            apple = apple.replace('&deg;', '°')
            apple = apple.replace('&quot;', '"')
            apple = apple.replace('\t', '')
            apple = apple.replace(',', '')
            apple = apple.replace('\n', '')
            apple = apple.replace('é', 'e')
            apple = apple.replace('ç', 'c')
            apple = apple.replace('è','e')
            apple = apple.replace('à', 'a')
            apple = apple.replace('ù', 'u')
            apple = apple.replace('ê','e') #il y a forcément un moyen plus simple
            tree = ET.fromstring(apple)
            a = tree.getchildren()
    
            annonces.append(a)
    
        pavel = []
        bar2 = pbar(len(annonces))
        for annonce in annonces:
            bar2.update(annonces.index(annonce))
            for j in range(len(annonce)):
                extracted_annonce, tags = xml_parser(paths_common, paths_take_all, annonce[j])
                p.append(extracted_annonce)
                
        write_to_csv(pavel, years[i], tags, years[i])


    

In [ ]:
write_years2csv(2005, 2014)

[                                                      ] N/A% (ETA:  --:--:--) 

181247 rows for year 2005


[                                                      ] N/A% (ETA:  --:--:--) 

200116 rows for year 2006


[                                                      ] N/A% (ETA:  --:--:--) 

...

[                                                      ] N/A% (ETA:  --:--:--) 

180415 rows for year 2008


[                                                      ] N/A% (ETA:  --:--:--) 

177523 rows for year 2009


[=================                                     ]  31% (ETA:   0:00:28) 

In [15]:
paths = glob.glob('rand/*.xml')
file = open(random.choice(paths), encoding='ISO-8859-1')
apple = file.read()
apple = apple.replace('&apos;', "'")
apple = apple.replace('&deg;', '°')
apple = apple.replace('&quot;', '"')
tree = ET.fromstring(apple)
a = tree.getchildren()


In [20]:
for key in tree.keys():
    print(tree.get(key))

20130110
11 juin 2013
BOMP C


In [133]:
print(len(a[26].findall('DONNEES/PROCEDURES/LOTS/NUM_LOT')))
reslit = list(a[26].find('DONNEES/PROCEDURES/LOTS').iter())

13


In [242]:
len(p[0])

1